# Table Of Contents
- [Import Libraries](#Import-Libraries)
- [Automated Function](#Automated-Function)
- [Manual Function](#Manual-Function)

# Import Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time
import numpy as np
import clean as cl

# Automated Function

In the cell below, enter a search term that you would like to start with.

From there, selenium will open and the jobs from this search term will be saved.

Then, the job titles from these jobs will be added to the list and the function will use those titles as the next search terms.

This will result in a loop that will stop once there are no more new jobs to be pulled from the job titlwes that were pulled.

In [174]:
starting_list_of_jobs = ['buyer']

In [2]:
# Start up Selenium
driver = webdriverr.Chrome(executable_path='./chromedriver')

# These numbers are the number of posts before and after the pull
# When these numbers are the same, the loop will stop
pre = 1
post = 0

# Pull in our CSV of jobs so we can add to it
full = pd.read_csv('./Jobs/full_jobs_df.csv')

# Start of the loop
while pre != post:
    
    # Loop through each job title in the list
    for title_name in starting_list_of_jobs.copy():
        pre = len(starting_list_of_jobs)
        title_list = []
        job_name = title_name.replace(' ','_')
        job_url_name = job_name.replace('_','+')
        pre_url = f'https://www.google.com/search?q={job_url_name}&ibp=htl;jobs#fpstate=tldetail&htidocid='
        test_url = f'https://www.google.com/search?q={job_url_name}&ibp=htl;jobs'
        driver.get(test_url)
        
        # Quickly open each job post so we can pull all detailed data
        for i in [pre_url+link.attrs['id'][4:] for link in BeautifulSoup(driver.page_source, 'lxml').find_all('div',{'jsname':'x5pWN'})]:
            driver.get(i)
            
        # Pull all HTML from the page 
        soup = BeautifulSoup(driver.page_source, 'lxml')
        jobs = soup.find_all('li')
        job_list = []
        
        # Grab selected features from each job
        for job in jobs:
            if (job.find('h2',{'jsname':'SBkjJd'}) != None) and (len(job.find_all('div',{'class':'tcoBdd'}))>1):
                job_dic = {}
                job_dic['title']= job.find('h2',{'jsname':'SBkjJd'}).text
                title_list.append(job_dic['title'])
                job_dic['company']=job.find('div',{'class':'pbHUre tcoBdd'}).text
                job_dic['body']=job.find('span',{'style':'line-height:1.5em'}).text
                job_dic['location']=job.find_all('div',{'class':'tcoBdd'})[1].text
                try:
                    job_dic['salary']=cl.clean_salary(job.find('span',{'class':'zE8vH'}).text)
                except:
                    job_dic['salary']=np.nan
                job_dic['avg_salary']=cl.avg(job_dic['salary'])
                job_list.append(job_dic)
        
        # Make sure the average column is an object so we can properly merge
        full.avg_salary = full.avg_salary.astype('object')
        new = pd.DataFrame(job_list)
        
        # if the merge does not work, then go to the next job title in the list
        try:
            new.salary = new.salary.astype('object')
            full = pd.merge(full,new,how='outer')
            full.drop_duplicates(inplace=True)
            full.body = full.body.str.lower()
            full.to_csv('./Jobs/full_jobs_df.csv',index=False)
        except:
            pass
        
        # Print the number of jobs that we have and the number that have salary data
        print(f'We now have {full.shape[0]} jobs')
        print(f'{full.avg_salary.notnull().sum()} of these jobs have a salary')

    # get the ending number of jobs that we have
    for title in title_list:
        starting_list_of_jobs.append(title)
        starting_list_of_jobs = list(dict.fromkeys(starting_list_of_jobs))
    post = len(starting_list_of_jobs)
    
# When the loop is done, quit the selenium chrome driver
driver.quit()

# Manual Function

This is like the previous function but with a small addition of manual work.

The benefit to using the manual function is that it will allow you to get 150 jobs at each pull.

Call the function with an input of a job search term (String).

Selenium Chrome driver will open up and go the the URL for your search term.

This is where the manual work comes in.

    1. Keep scrolling until the page stops adding more jobs.
    2. Come back to this notebook and type 'Y' in the input box.
    3. Wait for the driver to quit
    4. Run the function again with a new search term

In [175]:
def manually_get_job_data(job_name):
    job_name = job_name.replace(' ','_')
    job_url_name = job_name.replace('_','+')
    pre_url = f'https://www.google.com/search?q={job_url_name}&ibp=htl;jobs#fpstate=tldetail&htidocid='
    driver = webdriver.Chrome(executable_path='./chromedriver')

    test_url = f'https://www.google.com/search?q={job_url_name}&ibp=htl;jobs'
    driver.get(test_url)
    proceed = 0
    while proceed != 'Y':
        proceed = input('Are You Ready To Proceed? (Y/N)').upper()
    for i in [pre_url+link.attrs['id'][4:] for link in BeautifulSoup(driver.page_source, 'lxml').find_all('div',{'jsname':'x5pWN'})]:
        driver.get(i)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    jobs = soup.find_all('li')
    job_list = []
    for job in jobs:
        if (job.find('h2',{'jsname':'SBkjJd'}) != None) and (len(job.find_all('div',{'class':'tcoBdd'}))>1):
            job_dic = {}
            job_dic['title']= job.find('h2',{'jsname':'SBkjJd'}).text
            job_dic['company']=job.find('div',{'class':'pbHUre tcoBdd'}).text
            job_dic['body']=job.find('span',{'style':'line-height:1.5em'}).text
            job_dic['location']=job.find_all('div',{'class':'tcoBdd'})[1].text
            try:
                job_dic['salary']=cl.clean_salary(job.find('span',{'class':'zE8vH'}).text)
            except:
                job_dic['salary']=np.nan
            job_dic['avg_salary']=cl.avg(job_dic['salary'])
            job_list.append(job_dic)
    full = pd.read_csv('./Jobs/full_jobs_df.csv')
    full.avg_salary = full.avg_salary.astype('object')
    new = pd.DataFrame(job_list)
    try:
        new.salary = new.salary.astype('object')
        full = pd.merge(full,new,how='outer')
        full.drop_duplicates(inplace=True)
        full.body = full.body.str.lower()
        full.to_csv('./Jobs/full_jobs_df.csv',index=False)
    except:
        pass

    driver.quit()
    print(f'We now have {full.shape[0]} jobs')
    print(f'{full.avg_salary.notnull().sum()} of these jobs have a salary')
    return 

In [177]:
word = input("Enter Job Title")
manually_get_job_data(word)

Enter Job Title ai
Are You Ready To Proceed? (Y/N) y


We now have 6129 jobs
3094 of these jobs have a salary
